# Installing and importing dependencies

In [ ]:
#install these
# !pip install tensorflow==2.13.0    
# !pip install mediapipe==0.10.3
# !pip install scikit-learn==1.3.1
# !pip install opencv-python==4.9.0.80
# !pip install matplotlib==3.8.3

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Collecting keypoints from mediapipe holistics

In [ ]:
mp_holistic = mp.solutions.holistic  #we are bringing the holistic model of mideapipe
mp_drawing = mp.solutions.drawing_utils #we are also bringing the drawing utilities of mideapipe.

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(70,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(70,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(70,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(70,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #here we are reading the feed
        ret,frame = cap.read()
        
        #make detections
        image,results = mediapipe_detection(frame,holistic)
        print(results)
        
        #drawing landmarks on real time
        draw_landmarks(image,results)
        
        #now we are showing the feed to the screen
        cv2.imshow("OpenCV Feed",image)

        #breaking the loop if we press q.
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
results.left_hand_landmarks

In [ ]:
draw_landmarks(frame,results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Extracting keypoints value

In [ ]:
# results.pose_landmarks
# len(results.face_landmarks.landmark)
len(results.right_hand_landmarks.landmark)

In [ ]:
pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
extract_keypoints(results).shape
# 468*3+33*4+21*3+21*3

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0',result_test)

In [ ]:
np.load('0.npy')#load backup our array

# Folder setup for data Collection

In [ ]:
# path for exported data,numpy arrays
PATH = os.path.join("MP_Data")

# Actions that we are trying to detect
#
actions = ["haloo", "dannwa","mama", "obata" ,"puluwan", "suba" ,"udaasanak"]
# ,,, --

# total number of sequences that we are trying to collect which is 30 videos worth of data
no_sequences = 50

# 30 frames length of video
sequence_length = 50

# folder start
start_folder = 50

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(PATH,action,str(sequence)))
        except:
            pass

In [ ]:
os.listdir()

# COLLECTING KEYPOINTS FOR TRAINING AND TESTING

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                #here we are reading the feed
                ret,frame = cap.read()

                #make detections
                image,results = mediapipe_detection(frame,holistic)

                #drawing landmarks on real time
                draw_landmarks(image,results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX , 1, (255, 0,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0, 255,0), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(3000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Preprocessing our data

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        video = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(PATH, action, str(sequence), "{}.npy".format(frame_num)))
            video.append(res)
        sequences.append(video)
        labels.append(label_map[action])

In [ ]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

In [ ]:
X = np.array(sequences)
X.shape

In [ ]:
y = to_categorical(labels).astype(int)
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X_train.shape

# Building and training LSTM DL Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dropout
# from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

In [ ]:
# !pip install tensorboard==2.13.0
# load_ext tensorboard


In [ ]:
log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

In [ ]:
#instantiating our model
#here we are using media-pipe holistic and lstm model .
#less data is required,it is faster to train with 1.53 million parameters
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(50, 1662)))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
np.array(actions).shape

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tensorboard_callback])

In [ ]:
# tensorboard --logdir logs

In [ ]:
model.summary()

# Predicting result from test data

In [ ]:
result = model.predict(X_test)

In [ ]:
actions[np.argmax(result[0])]

In [ ]:
actions[np.argmax(y_test[0])]

# Saving weights for future

In [ ]:
tf.keras.models.save_model(
    model, "/Users/shehanliyanage/Research/MLAndFlask/finalModel.h5"
)

In [ ]:
# del model
from tensorflow.keras.models import load_model

In [ ]:
model.load_weights("finalModel.h5")

# Accuracy Calculation and Evaluation using Confusion matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)